In [2]:
import socket
from itertools import repeat
import time as t
from vnstock import *  # import all functions
import multiprocessing as mp
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
import numpy as np
import math
from dotenv import load_dotenv
load_dotenv("../.env")
stock_symbols = pd.read_csv("company.csv")["ticker"].tolist()
now = datetime.now()
today = str(now.date())

In [3]:
def retrieve_real_time_data(stock_symbols):
    df = pd.DataFrame()

    for symbol_index, stock_symbol in enumerate(stock_symbols):
        try:
            real_time_data = stock_historical_data(
                symbol=stock_symbol,
                start_date=today,
                end_date=today,
                resolution="15",
                type="stock",
                beautify=False,
                decor=False,
                source="DNSE",
            )
            if real_time_data is not None and not real_time_data.empty:
                df = df._append(real_time_data.iloc[-1])
        except Exception as e:
            print(f"Error processing stock symbol {stock_symbol}: {str(e)}")
            continue

    return df

In [4]:
def divide_list(input_list, num_sublists):
    sublist_length = math.ceil(len(input_list) / num_sublists)
    return [
        input_list[i : i + sublist_length]
        for i in range(0, len(input_list), sublist_length)
    ]

In [5]:
def collect_data_thread(num_of_thread):
    with ThreadPoolExecutor() as pool:
        return list(
            pool.map(retrieve_real_time_data, divide_list(stock_symbols, num_of_thread))
        )

In [10]:
def collect_data_process(num_of_thread):
    with ProcessPoolExecutor() as pool:
        return list(
            pool.map(retrieve_real_time_data, divide_list(stock_symbols, num_of_thread))
        )

In [10]:
def collect_data_normal():
    retrieve_real_time_data(stock_symbols=stock_symbols)
collect_data_normal()

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error processing stock symbol ROS: local variable 'df' referenced before assignment


In [11]:
collect_data_process(2)

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error processing stock symbol ROS: local variable 'df' referenced before assignment


[Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: []]

 Loại     | thời gian      |
| ------------- | ------------- |
| Chạy bình thường | 21 m 30 s |
| Thread 5 | 3m 58s |
| Thread 10 | 3m 5s |
| Thread 25 | 3m 24s |
| Thread 50 | 4m 5s |
| Process 2 | 6m 37s |